In [ ]:
# --- Cell 0: Run PyPetaKit5D Processing (Interactive) ---
%matplotlib widget

import opym
from pathlib import Path
import importlib
import sys

import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display

# Reload the opym module to make sure we have the new functions
try:
    importlib.reload(opym.petakit)
except AttributeError:
    pass
importlib.reload(opym)


# --- Create Interactive Widgets ---
print("1. Select the 'processed_tiff_series_split' directory.")
print("2. Click 'Run PetaKit Processing'")

# This chooser is set to select the processed directory
dir_chooser = FileChooser(
    path="/mmfs2/scratch/SDSMT.LOCAL/bscott/DataUpload",
    title="<b>Select Processed TIFF Directory (e.g., '.../processed_tiff_series_split'):</b>",
    show_only_dirs=True
)

run_button = widgets.Button(
    description="Run PetaKit Processing", button_style="success"
)
run_output = widgets.Output()

# --- Global variable to store the output path for the next cell ---
dsr_output_dir: Path | None = None

# --- Define the function to run on button click ---
def on_run_button_clicked(b):
    global dsr_output_dir
    
    with run_output:
        run_output.clear_output()
        
        if not dir_chooser.value:
            print("❌ ERROR: No directory selected. Please select a folder and try again.")
            return

        # Use a clear variable name for the selected path
        processed_dir = Path(dir_chooser.value)
        
        # --- Store the *output* path for the viewer cell ---
        # --- This path is now relative to the *parent* of the processed_dir ---
        # --- FIX: DSR output directory should be inside the processed_dir ---
        dsr_output_dir = processed_dir / "DSR"

        try:
            print(f"--- Running PetaKit5D processing for: {processed_dir.name} ---")
            
            # --- MODIFICATION: Call the function with specific OPM params ---
            opym.run_petakit_processing(
                processed_dir_path=processed_dir,
                
                # --- Parameters for OPM stage-scan ---
                xy_pixel_size=0.136,
                z_step_um=1.0,
                sheet_angle_deg=31.8,
                deskew=True,
                rotate=True,
                objective_scan=False,
                reverse_z=False,
                
                # You can override other params here, e.g.:
                # interp_method="cubic"
            )
            # --- END MODIFICATION ---
            
            print(f"\n✅ PetaKit5D job finished! Output is in: {dsr_output_dir.name}")
            print("   You can now run Cell 1 to view the results.")

        except FileNotFoundError as e:
            print(f"\n❌ ERROR: Could not find required directories or files.")
            print(f"  Details: {e}")
            print("  Please ensure you have selected the correct 'processed_tiff_series_split' folder.")
        except Exception as e:
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)

# --- Link button and display UI ---
run_button.on_click(on_run_button_clicked)
display(widgets.VBox([dir_chooser, run_button, run_output]))

In [ ]:
# --- Cell 1: Load Final Processed Data ---
# This cell loads the final deskewed + rotated TIFF series
# from the 'DSR' directory created in Cell 0.

import opym
from pathlib import Path
import importlib
import sys

import ipywidgets as widgets
from IPython.display import display

# Reload modules
try:
    importlib.reload(opym.dataloader)
    importlib.reload(opym.viewer)
    importlib.reload(opym)
    print("Reloaded opym.dataloader and opym.viewer")
except Exception as e:
    print(f"Note: Could not reload all modules: {e}")


# --- Global variable to hold viewer parameters ---
viewer_params = None

# --- Create Widgets ---
load_dsr_button = widgets.Button(
    description="Load & View Final DSR Data", button_style="primary"
)
load_dsr_output = widgets.Output()

# --- Define the function to run on button click ---
def on_load_dsr_button_clicked(b):
    global viewer_params  # Make sure we're assigning to the global var
    
    with load_dsr_output:
        load_dsr_output.clear_output()
        
        try:
            # Check that the previous cell has been run
            if dsr_output_dir is None:
                print("❌ ERROR: 'dsr_output_dir' not set. Please run Cell 0 first.")
                return

            if not dsr_output_dir.exists():
                 print(f"❌ ERROR: DSR directory not found at: {dsr_output_dir}")
                 print("   This is unexpected. Please re-run Cell 0.")
                 return

            print(f"--- Loading data from: {dsr_output_dir.name} ---")
            
            # 1. Call the data loader
            # ---
            # FIX: Unpack all 9 return values
            # ---
            (
                get_stack,
                t_min,
                t_max,
                c_min,
                c_max,
                z_max,
                y,
                x,
                base_name,
            ) = opym.load_tiff_series(dsr_output_dir)
            
            # 2. Store parameters in the global variable
            viewer_params = (get_stack, t_max, z_max, c_max, y, x)
            
            # ---
            # FIX: Update print statement
            # ---
            print(
                f"✅ Data loaded. Shape: T={t_min}-{t_max}, "
                f"Z={z_max + 1}, C={c_min}-{c_max}, Y={y}, X={x}"
            )
            print(f"   Base name: {base_name}")
            print("\nYou can now run the viewer cells below.")

        except FileNotFoundError as e:
            print(f"\n❌ ERROR: Could not load data.")
            print(f"  Details: {e}")
        except Exception as e:
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)

# --- Link button and display UI ---
load_dsr_button.on_click(on_load_dsr_button_clicked)
display(widgets.VBox([load_dsr_button, load_dsr_output]))

In [ ]:
# --- Cell 2: Launch the single-channel viewer ---
try:
    if viewer_params is None:
        raise NameError
    opym.single_channel_viewer(*viewer_params)
except NameError:
    print("❌ ERROR: 'viewer_params' not found. Please run Cell 1 to load data first.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

In [ ]:
# --- Cell 3: Launch the composite viewer ---
try:
    if viewer_params is None:
        raise NameError
    opym.composite_viewer(*viewer_params)
except NameError:
    print("❌ ERROR: 'viewer_params' not found. Please run Cell 1 to load data first.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")